In [1]:
import pandas as pd

In [4]:
df = pd.read_csv("./data/population.csv")

In [6]:
df.head()

,Entity,Code,Year,Population (historical)
0,Afghanistan,AFG,-10000,14737
1,Afghanistan,AFG,-9000,20405
2,Afghanistan,AFG,-8000,28253
3,Afghanistan,AFG,-7000,39120
4,Afghanistan,AFG,-6000,54166


In [8]:
df = df[df.Code != "OWID_WRL"]
df[df.Code == "OWID_WRL"]

,Entity,Code,Year,Population (historical)


In [50]:
small_df = df[:600]
medium_df = df[:6_000]
single_row_df = df[:1]

small_df.to_csv('./data/population-small.csv', header=True)
medium_df.to_csv('./data/population-medium.csv', header=True)
single_row_df.to_csv('./data/population-single-row.csv', header=True)

In [51]:
from typing import Union


def get_yoy_increase(group: pd.DataFrame) -> Union[float | None]:
    group = group.sort_values(by='Year')

    min_year_row = group.iloc[0]
    max_year_row = group.iloc[-1]

    year_diff = max_year_row['Year'] - min_year_row['Year']
    pop_diff = max_year_row['Population (historical)'] - min_year_row['Population (historical)']

    if year_diff != 0:
        return pop_diff / year_diff
    return None

agg_df = df.groupby(by='Entity').apply(get_yoy_increase).dropna().reset_index(name='YOY_increase')

# agg_df.describe()['YOY_increase']

In [44]:
test = pd.read_csv('./data/population-medium.csv')
len(test)

6000

In [66]:
import numpy as np
from tqdm import tqdm
from collections import defaultdict

In [68]:
def for_loop_pipeline(df: pd.DataFrame) -> list[float, float, float, float, float]:
    # Input: the dataframe from load_input()
    # Return a list of min, median, max, mean, and standard deviation
    entities = df["Entity"].unique()
    entities_dict = {}
    stats = defaultdict(dict)
    res = [0 for _ in range(len(entities))]

    for i, entity in enumerate(entities):
        entities_dict[entity] = i

    for _, entry in tqdm(df.iterrows(), desc="Iterating over rows"):
        entity = entry["Entity"]
        if entity not in stats:
            stats[entity]['max_year'] = entry["Year"]
            stats[entity]['min_year'] = entry["Year"]
            stats[entity]['max_population'] = entry["Population (historical)"]
            stats[entity]['min_population'] = entry["Population (historical)"]
            # stats[entity]['yoy_increase'] = 0
        else:
            if entry["Year"] > stats[entity]['max_year']:
                stats[entity]['max_year'] = entry["Year"]
                stats[entity]['max_population'] = entry["Population (historical)"]

            
            if entry["Year"] < stats[entity]['min_year']:
                stats[entity]['min_year'] = entry["Year"]
                stats[entity]['min_population'] = entry["Population (historical)"]

            if stats[entity]['min_year'] != stats[entity]['max_year']:
                res[entities_dict[entity]] = (stats[entity]['max_population'] - stats[entity]['min_population']) / (stats[entity]['max_year'] - stats[entity]['min_year'])

    # if max_year - min_year != 0:
    #     stats.append((newest_pop - oldest_pop) / (max_year - min_year))

    # Compute (min, median, max, mean, and standard deviation)
    return [np.min(res), np.median(res), np.max(res), np.mean(res), np.std(res)]

ans = for_loop_pipeline(df)

Iterating over rows: 59051it [00:06, 8785.64it/s] 


In [69]:
ans

[-36.68493150684932,
 859.3022540131415,
 46711357.31506849,
 330941.04907640134,
 3104595.5846900907]